In [1]:
import pandas as pd
import os
import numpy as np
import scipy
import empyrical as ep
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

In [2]:
#Risk-free Returns
relative_path = os.path.join('..', '..',"Data","Consolidated","TreasuryYield","TreasuryBill.csv")
r_f=pd.read_csv(relative_path,index_col=0)

In [3]:
def risk_return(df,forecast_horizon,strategy):
    profits=[]
    for i in range(0,df.shape[1]//2):
        profits.append(sum(df.iloc[:,i]))
    AnnualizedReturn=[]
    volatility=[]
    skewness=[]
    kurtosis=[]
    VaR=[]
    mdd=[]
    SharpeRatio=[]
    for i in range(df.shape[1]//2,df.shape[1]):
        #annualized return
        total_r=sum(df.iloc[:,i])
        n_years=len(df.iloc[:,i])/4
        ann_r=total_r/n_years
        #AnnualizedReturn.append(ann_r)
        #annualized excess return
        qrf=((((r_f.loc[df.index,forecast_horizon]/100)+1)**0.25)-1)
        simple_r=(np.exp(df.iloc[:,i])-1)
        excess_return_s=simple_r-qrf
        excess_return_log=np.log(1+excess_return_s)
        total_r_log=sum(excess_return_log)
        AnnualizedReturn.append(total_r_log/n_years)
        #volatility
        vol_q=np.std(df.iloc[:,i])
        vol_a=vol_q*np.sqrt(4)
        volatility.append(vol_a)
        #skewness
        skewness.append(scipy.stats.skew(df.iloc[:,i]))
        #kurtosis
        kurtosis.append(scipy.stats.kurtosis(df.iloc[:,i]))
        #Value at Risk
        confidence_level= 0.05
        var= scipy.stats.norm.ppf(confidence_level, ann_r,vol_a)
        VaR.append(var)
        #Maximum Drawdown
        simple_r=(np.exp(df.iloc[:,i])-1)
        mdd.append(ep.max_drawdown(simple_r))
        #Sharpe Ratio
        #qrf=((((r_f.loc[df.index,forecast_horizon]/100)+1)**0.25)-1)
        #excess_return=simple_r-qrf
        xr=np.mean(excess_return_s)
        sd=np.std(simple_r)
        SharpeRatio.append(xr/sd)
    rr_dict={
        #"Profit (in USD)":profits,
        "Log-ER (ann.)":AnnualizedReturn,
        "Volatility (ann.)": volatility,
        "Skewness":skewness,
        "Kurtosis":kurtosis,
        "VaR (95%)": VaR,
        "Maximum Drawdown":mdd,
        "Sharpe Ratio": SharpeRatio
    }
    rr_df=pd.DataFrame(rr_dict)
    if strategy=="A":
        rr_df.index=["No-Change(g)","Future Implied Yield","Forward","Expert Forecasts","Individual Forecasts"]
    elif strategy=="B":
         rr_df.index=["No-Change(F)","No-Change(g)","Forward","Expert Forecasts","Individual Forecasts"]
    elif strategy=="C":
         rr_df.index=["No-Change(g)","Forward","Expert Forecasts","Individual Forecasts"]
    rr_df.loc(axis=0)["Number of Observations"]=df.count()[0]
    return(rr_df)

# Return Graphs
The Graphs below show how 100$ invested into the strategies would have performed over time

In [4]:
def MDD_df_calc(returns,forecast_type):
    returns.index=pd.to_datetime(returns.index)
    init=100
    HWM=[100]
    values=[100]
    MDD=[]
    DD=[]
    for i in returns[forecast_type]:
        r_simp=np.exp(i)
        init=init*r_simp
        values.append(init)
        HWM.append(max(init,HWM[-1]))
        DD.append(1-init/max(HWM))
        MDD.append(max(DD))
    last_q=pd.Timestamp(returns.index[-1])
    next_quarter_end=last_q.to_period("Q")+1
    next_quarter_end_date=next_quarter_end.to_timestamp(how="end")
    dates=list(returns.index)
    dates.append(next_quarter_end_date)
    MDD_df=pd.DataFrame({"HWM":HWM,"Values":values},index=dates)
    return(MDD_df)

In [5]:
#function to export formatted data to excel

def file_saver(df,relative_path):
    with pd.ExcelWriter(relative_path, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name='Sheet1', index=True)
        
        workbook = writer.book
        worksheet = writer.sheets['Sheet1']
        
        # Apply formatting
        format_float = workbook.add_format({'num_format': '0.00'})
        format_percent = workbook.add_format({'num_format': '0.00%'})
        format_three_decimals = workbook.add_format({'num_format': '0.000'})
        format_int= workbook.add_format({'num_format': '0'})

        n=len(df)
        
        # Apply the formats to the appropriate columns
        #worksheet.set_column('A:A', None, format_float)  # "Profit (in USD)"
        worksheet.set_column('B:B', None, format_percent) # "Log-Returns (ann.)"
        worksheet.set_column('C:C', None, format_percent)  # "Volatility (ann.)"
        worksheet.set_column('D:D', None, format_three_decimals)  # "Skewness"
        worksheet.set_column('E:E', None, format_three_decimals)  # "Kurtosis"
        worksheet.set_column('F:F', None, format_percent)  # "VaR (95%)"
        worksheet.set_column('G:G', None, format_percent)  # "Maximum Drawdown"
        worksheet.set_column('H:H', None, format_three_decimals)  # "Sharpe Ratio"
        worksheet.set_row(n, None, format_int)  # "Number of Observations"

In [6]:
strategies=["A","B","C"]
forecast_horizons=["3M","6M","1Y"]
Instrument=["ETF","Futures"]
Yields=[2,10]

for strategy in strategies:
    for forecast_horizon in forecast_horizons:
        for instrument in Instrument:
            for yield_ in Yields:
                if forecast_horizon=="1Y":
                    f_h_name="12M"
                else:
                    f_h_name=forecast_horizon
                relative_path = os.path.join('..', '..',"Data","Results","TradingStrategies","Alternative","Strategy"+strategy,instrument,"returnsA"+str(yield_)+"Y"+f_h_name+".csv")
                df=pd.read_csv(relative_path,index_col=0)
                return_df=risk_return(df,forecast_horizon,strategy)
                relative_path = os.path.join('..', '..',"Data","Results","RiskReturn","Alternative","Strategy"+strategy,instrument,"RiskReturnA"+str(yield_)+"Y"+f_h_name+".xlsx")
                file_saver(return_df,relative_path)
                #return_df.to_excel(relative_path)

In [7]:
def excess_return_calc(df,forecast_horizon,r_f):
    r_f.index=pd.to_datetime(r_f.index)
    qrf=((((r_f.loc[df.index,forecast_horizon]/100)+1)**0.25)-1)
    simple_r=np.exp(df)-1
    excess_return_s=simple_r.sub(qrf,axis=0)
    excess_return_log=np.log(1+excess_return_s)
    return (excess_return_log)

In [8]:
def graph_creator(df,relative_path):
    df.index=pd.to_datetime(df.index)
    n=df.shape[1]//2
    #only keep return data
    new_df=df.iloc[:,-n:]
    #add first quarter with 0 returns
    new_date=df.index[0]-pd.offsets.QuarterEnd(n=1)
    new_df.loc(axis=0)[new_date]=0
    new_df=new_df.sort_index()
    new_df.columns=[column.replace(" - Return", "") for column in new_df.columns]
    new_df.rename(columns={'Future Implied Yield':'Futures-Implied Yield'}, inplace=True)
    #create the cumulative sum graph
    ax = new_df.cumsum().plot(figsize=(13, 7))
    plt.axhline(color="black")
    plt.legend()
    matplotlib.style.use("seaborn-paper")
    
    # Format y-axis as percentages
    ax.yaxis.set_major_formatter(mticker.PercentFormatter(1.0))
    
    plt.savefig(relative_path,bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
strategies=["A","B","C"]
forecast_horizons=["3M","6M","1Y"]
Instrument=["ETF","Futures"]
Yields=[2,10]

for strategy in strategies:
    for forecast_horizon in forecast_horizons:
        for instrument in Instrument:
            for yield_ in Yields:
                if forecast_horizon=="1Y":
                    f_h_name="12M"
                else:
                    f_h_name=forecast_horizon
                relative_path = os.path.join('..', '..',"Data","Results","TradingStrategies","Alternative","Strategy"+strategy,instrument,"returnsA"+str(yield_)+"Y"+f_h_name+".csv")
                df=pd.read_csv(relative_path,index_col=0)
                relative_path = os.path.join('..', '..',"Data","Results","RiskReturn","Alternative","Strategy"+strategy,instrument,"CumulativeReturn"+str(yield_)+"Y"+f_h_name+".png")
                graph_creator(df,relative_path)
                #return_df.to_excel(relative_path)